# The FDR function in Pyleoclim

This notebook presents how to use the `fdr` function in Pyleoclim.

## Basic idea of FDR

Quote from [Ventura et al. (2004)](https://journals.ametsoc.org/doi/full/10.1175/3199.1): "A single test performed at signiﬁcance level $\alpha$ has probability $\alpha$ of rejecting the null hypothesis when it is in fact true. Hence if $n$ such tests are performed when all $n$ null hypotheses are true (the collective null hypothesis), then the average number of tests for which the null is falsely rejected is $n\alpha$. For example, with $\alpha = 5\%$, testing for a trend at 1000 locations at which no change really occurred would yield 50 signiﬁcant locations on average; this is unacceptably high."

To overcome this disadvantage, there have been some alternatives, among which one is False Discovery Rate (FDR), originally by Benjamini & Hochberg (1995) (BH95 hereafter). There are other variants such as Yekutieli & Benjamini (1999, 2001), and Ventura et al. (2004).

In a multiple testing procedure, we have a list of $p$-values.
With the traditional false positive test, the significance level $\alpha=0.05$ is refered as the false positive rate (FPR), and we reject null hypothesis $H_0$ at all locations $p_i < \alpha$.

In the original BH95 method, the nominal FDR, denoted by $q$, is defined as the rate we are willing to allow of false rejections out of all rejections, and is usually set to $5\%$. The BH95 procedure rejects null hypothesis $H_0$ at all locations $i$ for which $p_i \le p_k$, where $k = \max_{i=0,...,n} \{i: p_{(i)}\le q i/n \}$.

## The function in Pyleoclim

In [1]:
from pyleoclim.utils.correlation import fdr

?fdr

Signature: fdr(pvals, qlevel=0.05, method='original', adj_method=None, adj_args={})
Docstring:
Determine significance based on the FDR approach (translated from fdr.R by Dr. Chris Paciorek)

Args
----

pvals : list or array
    A vector of p-values on which to conduct the multiple testing.

qlevel : float
    The proportion of false positives desired.

method : {'original', 'general'}
    Method for performing the testing.
    - 'original' follows Benjamini & Hochberg (1995);
    - 'general' is much more conservative, requiring no assumptions on the p-values (see Benjamini & Yekutieli (2001)).
    'original' is recommended, and if desired, using 'adj_method="mean"' to increase power.

adj_method: {'mean', 'storey', 'two-stage'}
    Method for increasing the power of the procedure by estimating the proportion of alternative p-values.
    - 'mean', the modified Storey estimator in Ventura et al. (2004)
    - 'storey', the method of Storey (2002)
    - 'two-stage', the iterative approach 

### Example case 1

First let's generate a list of $p$-values with size 10 to emulate 10 tests.

In [74]:
import numpy as np

np.random.seed(2333)
pval_list = np.random.random(10)/10
print('p-value list:\n', pval_list)

p-value list:
 [0.05297776 0.08944875 0.02423382 0.09269607 0.01918608 0.03875847
 0.02873022 0.07296763 0.04393807 0.03032502]


In [64]:
fdr_res = fdr(pval_list, method='original')
print(fdr_res)

None


In the case above, we get `None`, which means that there are no significant tests.
Now let's test another case below.
Note that the 3rd test has p-value equals to 0.024, which will be treated as significant in traditional false positive test where the significance level $\alpha=0.05$.

### Example case 2

In [76]:
np.random.seed(2333)
pval_list = np.random.random(10)/15
print('p-value list:\n', pval_list, '\n')

fdr_res = fdr(pval_list, method='original')
print(fdr_res, '\n')

print('Index\tp-value')
for idx in fdr_res:
    print(f'{idx}\t{pval_list[idx]}')

p-value list:
 [0.03531851 0.0596325  0.01615588 0.06179738 0.01279072 0.02583898
 0.01915348 0.04864508 0.02929205 0.02021668] 

[2 4 5 6 8 9] 

Index	p-value
2	0.01615588065179053
4	0.01279071873650784
5	0.02583898106901107
6	0.019153480900810392
8	0.029292048677502642
9	0.020216678222839013


In this 2nd case, the `fdr` function returns a list of indices, which indicates the significant ones among the 10 tests.
Note that the 1st test has p-value equals to 0.035, which will be treated as significant in traditional false positive test where the significance level $\alpha=0.05$.

In [12]:
import numpy as np

def freq_vector_log_fix(ts, nfreq=None):
    ''' Return the frequency vector based on logspace

    Args
    ----

    ts : array
        time axis of the time series

    nv : int
        the parameter that controls the number of freq points

    Returns
    -------

    freq : array
        the frequency vector

    '''

    nt = np.size(ts)
    dt = np.median(np.diff(ts))
    fs = 1 / dt
    if nfreq is None:
        nfreq = nt//10 + 1

    fmin = 2/(np.max(ts)-np.min(ts))
    fmax = fs/2
    start = np.log2(fmin)
    stop = np.log2(fmax)

    freq = np.logspace(start, stop, nfreq, base=2)

    return freq

In [13]:
t = np.array([   20.0035 ,    22.1215 ,    38.075  ,    95.535  ,   152.995  ,
         210.455  ,   267.915  ,   317.284  ,   342.38   ,   367.476  ,
         392.572  ,   417.668  ,   436.49   ,   445.56   ,   481.84   ,
         518.12   ,   554.4    ,   600.1084 ,   674.102  ,   748.0956 ,
         822.0892 ,   896.0828 ,   960.74885,   997.43225,  1034.11565,
        1070.79905,  1107.48245,  1139.6092 ,  1158.066  ,  1176.5228 ,
        1194.9796 ,  1213.4364 ,  1235.33615,  1267.56475,  1299.79335,
        1332.02195,  1364.25055,  1389.48485,  1393.73625,  1397.98765,
        1402.23905,  1406.49045,  1423.3671 ,  1478.1195 ,  1532.8719 ,
        1587.6243 ,  1642.3767 ,  1708.56155,  1809.04375,  1909.52595,
        2010.00815,  2110.49035,  2196.4424 ,  2238.804  ,  2281.1656 ,
        2323.5272 ,  2365.8888 ,  2398.76565,  2403.18825,  2407.61085,
        2412.03345,  2416.45605,  2436.3877 ,  2502.8465 ,  2635.7641 ,
        2702.2229 ,  2760.66655,  2795.06475,  2829.46295,  2863.86115,
        2898.25935,  2990.991  ,  3044.5374 ,  3098.0838 ,  3151.6302 ,
        3210.1285 ,  3283.4825 ,  3356.8365 ,  3430.1905 ,  3503.5445 ,
        3574.9425 ,  3640.4725 ,  3706.0025 ,  3771.5325 ,  3837.0625 ,
        3899.113  ,  3950.725  ,  3950.725  ,  4002.337  ,  4053.949  ,
        4105.561  ,  4149.7155 ,  4171.4975 ,  4193.2795 ,  4215.0615 ,
        4236.8435 ,  4256.2635 ,  4268.5975 ,  4280.9315 ,  4305.5995 ,
        4330.502  ,  4393.11   ,  4455.718  ,  4518.326  ,  4580.934  ,
        4631.8905 ,  4647.8925 ,  4663.8945 ,  4679.8965 ,  4695.8985 ,
        4715.7475 ,  4747.1375 ,  4778.5275 ,  4809.9175 ,  4841.3075 ,
        4865.534  ,  4868.27   ,  4871.006  ,  4873.742  ,  4876.478  ,
        4894.5535 ,  4958.6475 ,  5022.7415 ,  5086.8355 ,  5150.9295 ,
        5203.1745 ,  5219.8725 ,  5236.5705 ,  5253.2685 ,  5269.9665 ,
        5282.49   ,  5303.33375,  5386.70875,  5470.08375,  5553.45875,
        5652.898  ,  5800.53   ,  5948.162  ,  6095.794  ,  6243.426  ,
        6381.5285 ,  6491.0425 ,  6600.5565 ,  6710.0705 ,  6819.5845 ,
        6902.324  ,  6904.74   ,  6907.156  ,  6909.572  ,  6911.988  ,
        6916.2685 ,  6926.1425 ,  6936.0165 ,  6955.7645 ,  6970.3145 ,
        6998.8925 ,  7027.4705 ,  7056.0485 ,  7133.416  ,  7242.84   ,
        7352.264  ,  7461.688  ,  7571.112  ,  7664.46   ,  7754.7    ,
        7799.82   ,  7844.94   ,  7878.8365 ,  7879.0625 ,  7879.2885 ,
        7879.5145 ,  7887.301  ,  7916.865  ,  7946.429  ,  7975.993  ,
        8033.051  ,  8054.335  ,  8075.619  ,  8118.187  ,  8147.92   ,
        8203.     ,  8258.08   ,  8313.16   ,  8368.24   ,  8417.4805 ,
        8449.2025 ,  8480.9245 ,  8512.6465 ,  8544.3685 ,  8570.6775 ,
        8580.7475 ,  8590.8175 ,  8600.8875 ,  8608.455  ,  8612.521  ,
        8628.605  ,  8644.689  ,  8660.773  ,  8676.857  ,  8695.969  ,
        8724.165  ,  8752.361  ,  8808.753  ,  8838.27   ,  8871.75   ,
        8905.23   ,  8938.71   ,  8972.19   ,  8998.5955 ,  9003.7775 ,
        9008.9595 ,  9019.3235 ,  9036.2055 ,  9140.1695 ,  9192.1515 ,
        9244.1335 ,  9310.56   ,  9332.512  ,  9354.464  ,  9376.416  ,
        9423.3565 ,  9545.2625 ,  9667.1685 ,  9789.0745 , 10019.7615 ,
       10158.5735 , 10227.9795 , 10297.3855 , 10351.409  , 10359.285  ,
       10367.161  , 10375.037  , 10382.913  , 10420.735  , 10446.267  ,
       10452.65   , 10452.65   ])

In [14]:
from pyleoclim.utils.wavelet import freq_vector_log, freq_vector_nfft

In [15]:
freq_log = freq_vector_log(t)
freq_fix = freq_vector_log_fix(t)
freq_nfft = freq_vector_nfft(t)
print(1/freq_log)
print(1/freq_fix)
print(1/freq_nfft)

[237.         224.32707734 212.33180433 200.97794553 190.23120308
 180.0591131  170.43094764 161.31762182 152.69160602 144.52684266
 136.79866756 129.48373535 122.559949   116.00639307 109.8032705
 103.93184284  98.37437362  93.11407476  88.13505591  83.42227638
  78.96149977  74.73925091  70.74277519  66.96      ]
[5216.32325    4316.4153989  3571.75753934 2955.56630696 2445.67894059
 2023.75614661 1674.62248335 1385.7205407  1146.65928352  948.84031366
  785.14860845  649.69661225  537.61247671  444.86483331  368.11779579
  304.61097716  252.06020591  208.57537045  172.59243679  142.81719445
  118.17870707   97.79079374   80.92015539   66.96      ]
[          inf 7901.28       3950.64       2633.76       1975.32
 1580.256      1316.88       1128.75428571  987.66        877.92
  790.128       718.29818182  658.44        607.79076923  564.37714286
  526.752       493.83        464.78117647  438.96        415.85684211
  395.064       376.25142857  359.14909091  343.53391304  329.22
  31

/Users/fzhu/Apps/miniconda3/envs/py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in true_divide
  
